<a href="https://colab.research.google.com/github/TK-Problem/Python-mokymai/blob/master/ML_ir_AI/AI_portreto_generatorius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dreambooth AI sugeneruoto portreto projektas

Šis projektas buvo adaptuotas pagal buildspace [AI Avatar project](https://buildspace.so/builds/ai-avatar) . Rekomenduoju užsiregistruoti ir savarankiškai pereiti per šį projektą. Tai padaryti galite nukeliavę į šią [nuorodą](https://buildspace.so/p/build-ai-avatars).

Builspace sugeneravo šį google colab norebook'ą iš kito projekto [Dreambooth diffusers by Shivam Shrirao](https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth).  Apache-2.0 license observed.

# I. žingsnis - Prisijungti prie virtulios mašinos

Šis projektas skirtas leisti ant google nemokamai prieinamų grafinių kortų (GPU). Pasitikrinkite per `Edit` -> `Notebook settings`, kad yra pasirinkta GPU.


![image info](https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-1-4842-4470-8_7/MediaObjects/463852_1_En_7_Fig4_HTML.jpg)



In [ ]:
#@markdown Ši kodo eilutė yra skirta paordyti, kokią grafinę kortą ir kiek laisvų RAM skyrė google
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# II. žingsnis - Įrašyti bibliotekas
Žemiau esantis kodo blokas yra skirtas įrašyti Python bibliotekoms į virtualią aplinką.

In [ ]:
!wget -q https://raw.githubusercontent.com/buildspace/diffusers/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.15.0 transformers ftfy bitsandbytes==0.35.0 gradio natsort

# III. žingsnis - Prisijungti prie HuggingFace 🤗

Reikia susikurti paskyrą HuggingFace, kad galima būtų iš atsisiųsti `Stable Diffusion` paruoštą modelį. Nueikit į [HuggingFace](huggingface.co?ref=buildspace) ir susikurkite nemokamą paskyrą.

Kai susikurtsite paskyrą, reikia susigeneruoti `write` žetoną [tokens](https://huggingface.co/settings/tokens) puslapį.

![token create](https://hackmd.io/_uploads/BJ1wdTNqi.png)

Šiam projekte naudojame `Stable Diffusion v1.5` sukurtą `Runway`, jo licenziją galite peržiūrėti čia [modelio aprašymas](https://huggingface.co/runwayml/stable-diffusion-v1-5).

In [ ]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

# IV. žingsnis - įsirašyti xformers sukompiliuota biblioteka
xformers yra papildoma biblioteka, kurios reikia dirbant su kalbos interpretavimu ir teksto clasifikavimu.

In [ ]:
%pip install -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
# šis kodas buvo sukompiliuotas ant Tesla T4.

# jei Jums kodas neveikia, tuomet nuimkite komentarus nuo žemiau esančios eilutės
# (CTRL + /) ir įrašykite nekompiliuotą modelį modelį (gali užtrukti kokias 40 min)
# %pip install git+https://github.com/facebookresearch/xformers@4c06c79#egg=xformers